# 翻訳比較


In [73]:
import logging
import time
import json
import os

from dotenv import load_dotenv
import requests

import cohere
import anthropic
import openai
from openai import OpenAI

In [74]:
load_dotenv()

CLAUDE_API_KEY = os.getenv("CLAUDE_APIKEY")
COHERE_API_KEY = os.getenv("COHERE_APIKEY")
OPENAI_API_KEY = os.getenv("OPENAI_APIKEY")

In [58]:
translation_query = lambda text: f"""以下の英語テキストを日本語に翻訳してください。
翻訳は自然で流暢な日本語にし、原文のニュアンスと意味を正確に保ってください。
   
英語テキスト:
{text}

日本語翻訳:"""

summarization_query = lambda text: f"""以下のテキストを日本語で400字程度に要約してください。
翻訳は自然で流暢な日本語にし、原文のニュアンスと意味を正確に保ってください。
   
オリジナルテキスト:
{text}

日本語要約:"""

class AIModelClient:

    def translate(self, text: str) -> Dict:
        start_time = time.time()
    
        try:
            translation, model = self._concrete_translate(text)
            elapsed_time = time.time() - start_time
            
            return {
                "translation": translation,
                "time": elapsed_time,
                "model": model,
                "success": True,
                "error": None
            }
        except Exception as e:
            return {
                "translation": None,
                "time": time.time() - start_time,
                "model": model,
                "success": False,
                "error": str(e)
            }

    def summarize(self, text: str) -> Dict:
        start_time = time.time()
    
        try:
            translation, model = self._concrete_summarize(text)
            elapsed_time = time.time() - start_time
            
            return {
                "translation": translation,
                "time": elapsed_time,
                "model": model,
                "success": True,
                "error": None
            }
        except Exception as e:
            return {
                "translation": None,
                "time": time.time() - start_time,
                "model": model,
                "success": False,
                "error": str(e)
            }


class ClaudeClient(AIModelClient):
    def __init__(self, api_key):
        self.client = anthropic.Anthropic(api_key=api_key)

    def _concrete_translate(self, text: str, model: str = "claude-sonnet-4-5") -> Dict:
        message = self.client.messages.create(
            model=model,
            max_tokens=4096,
            messages=[
                {
                    "role": "user",
                    "content": translation_query(text)
                }
            ]
        )        
        return message.content[0].text, model

    def _concrete_summarize(self, text: str, model: str = "claude-sonnet-4-5") -> Dict:
            message = self.client.messages.create(
                model=model,
                max_tokens=4096,
                messages=[
                    {
                        "role": "user",
                        "content": summarization_query(text)
                    }
                ]
            )            
            return message.content[0].text, model

class CohereClient(AIModelClient):
    def __init__(self, api_key):
        self.client = cohere.Client(api_key=api_key)

    def _concrete_translate(self, text: str, model: str = "command-a-translate-08-2025") -> Dict:
        response = self.client.chat(
            model=model,
            message=translation_query(text),
            temperature=0.3
        )
        return response.text, model
    
    def _concrete_summarize(self, text: str, model: str = "command-a-03-2025") -> Dict:
        response = self.client.chat(
            model=model,
            message=summarization_query(text),
            temperature=0.3
        )            
        return response.text, model

class OpenAIClient:
    def __init__(self, api_key):
        self.client = OpenAI(api_key=api_key)

    def _concrete_translate(self, text: str, model: str = "gpt-4o") -> Dict:
        response = self.client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": "あなたは優秀な翻訳者です。英語を自然で流暢な日本語に翻訳してください。"
                },
                {
                    "role": "user",
                    "content": translation_query(text)
                }
            ],
            temperature=0.3
        )        
        return response.choices[0].message.content, model

    def _concrete_summarize(self, text: str, model: str = "gpt-4o") -> Dict:
        response = self.client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": "あなたは優秀な編集者です。自然で流暢な日本語に要約してください。"
                },
                {
                    "role": "user",
                    "content": summarization_query(text)
                }
            ],
            temperature=0.3
        )            
        return response.choices[0].message.content, model

claude_client = ClaudeClient(CLAUDE_API_KEY)
cohere_client = CohereClient(COHERE_API_KEY)
openai_client = OpenAIClient(OPENAI_API_KEY)

In [75]:
# Project Gutenberg, https://www.gutenberg.org/ebooks/12096
TEXT_URL = 'https://www.gutenberg.org/cache/epub/12096/pg12096.txt'
TEXT_FILE = 'data/I.Nitobe_Bushido.txt'
if not os.path.exists(TEXT_FILE):
    res = requests.get(TEXT_URL)
    with open(TEXT_FILE, "w") as f:
        f.write(res.text)
    
with open(TEXT_FILE) as f:
    text = f.read()

print(f"text size: {len(text)}")

text size: 217583


In [76]:
print(json.dumps(claude_client.summarize(text), indent=2, ensure_ascii=False))
print(json.dumps(cohere_client.summarize(text), indent=2, ensure_ascii=False))
#print(openai_client.summarize(text))

{
  "translation": "# 『武士道―日本の魂』要約\n\n新渡戸稲造による本書は、日本の武士階級の倫理規範である武士道について解説した作品である。武士道は成文化された法典ではなく、何世紀にもわたる軍事的伝統から生まれた道徳的原則の体系である。その起源は封建制度にあり、仏教、神道、儒教の教えから影響を受けている。\n\n武士道の核心的な徳目には、義(正義)、勇(勇気)、仁(慈悲)、礼(礼儀)、誠(誠実)、名誉、忠義が含まれる。これらの原則は武士の教育と訓練を導き、剣は武士の魂の象徴とされた。武士道は女性にも影響を与え、家庭内での強さと献身を奨励した。\n\n著者は切腹や仇討ちといった制度を検討し、それらが単なる野蛮な慣習ではなく、名誉と正義という武士道の価値観に根ざした儀式であったことを説明する。武士道の影響は武士階級を超えて広がり、日本国民全体の道徳的基準を形成した。近代化により武士道は衰退しつつあるが、その精神は日本文化に深く刻まれ、国民性を形作り続けている。新渡戸は、武士道の最良の要素がキリスト教の愛の理念と融合し、より普遍的な倫理体系へと進化する可能性を示唆している。",
  "time": 13.58290982246399,
  "model": "claude-sonnet-4-5",
  "success": true,
  "error": null
}
{
  "translation": "新渡戸稲造の『武士道』は、日本の武士道精神を倫理体系として解説した書籍である。武士道は、忠義、勇気、仁義、礼節、誠実、名誉などの徳目を重んじる騎士道精神であり、日本の封建制度下で育まれた。その精神は、仏教の無常観や忍耐、神道の神国思想や忠君愛国、儒教の五常の思想など、様々な思想的影響を受けている。\n\n武士道は単なる戦闘技術ではなく、日常生活における行動規範であり、武士の精神的支柱であった。正義、勇気、仁愛、礼儀、誠実、名誉などの徳目を重んじ、自己犠牲や責任感、節制などを求めた。特に忠義は武士道の中核をなす徳目であり、主君への絶対的な忠誠が求められた。\n\n武士道は教育や訓練を通じて武士に叩き込まれ、厳しい規律と自制が求められた。また、切腹や仇討ちなどの制度も武士道の精神を反映している。武士道は剣を魂とし、剣術は武士の重要な技術であった。